In [1]:
!pip install pyspark
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=733ecac2e09051967bdb1152c6f2433de0dd6180a6d79a1374116bb31a3bb577
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
mkdir linkage

In [3]:
cd linkage

/content/linkage


In [4]:
!curl -L -o donation.zip https://bit.ly/1Aoywaq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   163  100   163    0     0   1265      0 --:--:-- --:--:-- --:--:--  1273
100 53.8M    0 53.8M    0     0  31.1M      0 --:--:--  0:00:01 --:--:-- 55.9M


In [5]:
!unzip donation.zip

Archive:  donation.zip
 extracting: block_10.zip            
 extracting: block_1.zip             
 extracting: block_2.zip             
 extracting: block_3.zip             
 extracting: block_4.zip             
 extracting: block_5.zip             
 extracting: block_6.zip             
 extracting: block_7.zip             
 extracting: block_8.zip             
 extracting: block_9.zip             
  inflating: documentation           
  inflating: frequencies.csv         


In [6]:
!unzip 'block_*.zip'

Archive:  block_7.zip
  inflating: block_7.csv             

Archive:  block_8.zip
  inflating: block_8.csv             

Archive:  block_10.zip
  inflating: block_10.csv            

Archive:  block_3.zip
  inflating: block_3.csv             

Archive:  block_2.zip
  inflating: block_2.csv             

Archive:  block_1.zip
  inflating: block_1.csv             

Archive:  block_5.zip
  inflating: block_5.csv             

Archive:  block_4.zip
  inflating: block_4.csv             

Archive:  block_6.zip
  inflating: block_6.csv             

Archive:  block_9.zip
  inflating: block_9.csv             

10 archives were successfully processed.


In [7]:
prev = spark.read.csv("/content/linkage/block*.csv")
prev

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string, _c4: string, _c5: string, _c6: string, _c7: string, _c8: string, _c9: string, _c10: string, _c11: string]

In [8]:
prev.show(2)

+----+----+------------+------------+------------+------------+-------+------+------+------+-------+--------+
| _c0| _c1|         _c2|         _c3|         _c4|         _c5|    _c6|   _c7|   _c8|   _c9|   _c10|    _c11|
+----+----+------------+------------+------------+------------+-------+------+------+------+-------+--------+
|id_1|id_2|cmp_fname_c1|cmp_fname_c2|cmp_lname_c1|cmp_lname_c2|cmp_sex|cmp_bd|cmp_bm|cmp_by|cmp_plz|is_match|
|3148|8326|           1|           ?|           1|           ?|      1|     1|     1|     1|      1|    TRUE|
+----+----+------------+------------+------------+------------+-------+------+------+------+-------+--------+
only showing top 2 rows



In [10]:
parsed = spark.read.option("header", "true").option("nullValue", "?").option("inferSchema", "true").csv("/content/linkage/block*.csv")

In [11]:
parsed.printSchema()

root
 |-- id_1: integer (nullable = true)
 |-- id_2: integer (nullable = true)
 |-- cmp_fname_c1: double (nullable = true)
 |-- cmp_fname_c2: double (nullable = true)
 |-- cmp_lname_c1: double (nullable = true)
 |-- cmp_lname_c2: double (nullable = true)
 |-- cmp_sex: integer (nullable = true)
 |-- cmp_bd: integer (nullable = true)
 |-- cmp_bm: integer (nullable = true)
 |-- cmp_by: integer (nullable = true)
 |-- cmp_plz: integer (nullable = true)
 |-- is_match: boolean (nullable = true)



In [12]:
from pyspark.sql.types import *


schema = StructType([
    StructField("id_1", IntegerType(), False),
    StructField("id_2", IntegerType(), False),
    StructField("cmp_fname_c1", DoubleType(), False)])

spark.read.schema(schema).csv("/content/linkage/block*.csv").printSchema()

root
 |-- id_1: integer (nullable = true)
 |-- id_2: integer (nullable = true)
 |-- cmp_fname_c1: double (nullable = true)



In [13]:
schema = "id_1 INT, id_2 INT, cmp_fname_c1 DOUBLE"

spark.read.schema(schema).csv("/content/linkage/block*.csv").printSchema()

root
 |-- id_1: integer (nullable = true)
 |-- id_2: integer (nullable = true)
 |-- cmp_fname_c1: double (nullable = true)



In [15]:
d1 = spark.read.format("json").load("/content/json_files/Previsao.json")

In [16]:
d1

DataFrame[_corrupt_record: string]

In [17]:
d2 = spark.read.json("/content/json_files/Previsao.json")

In [18]:
d2

DataFrame[_corrupt_record: string]

In [21]:
d1.write.format("parquet").save("/content/parquet_files/Previsao.json")

AnalysisException: Since Spark 2.3, the queries from raw JSON/CSV files are disallowed when the
referenced columns only include the internal corrupt record column
(named _corrupt_record by default). For example:
spark.read.schema(schema).csv(file).filter($"_corrupt_record".isNotNull).count()
and spark.read.schema(schema).csv(file).select("_corrupt_record").show().
Instead, you can cache or save the parsed results and then send the same query.
For example, val df = spark.read.schema(schema).csv(file).cache() and then
df.filter($"_corrupt_record".isNotNull).count().

In [23]:
d1.write.parquet("/content/parquet_files/Previsao.parquet")

AnalysisException: [PATH_ALREADY_EXISTS] Path file:/content/parquet_files/Previsao.parquet already exists. Set mode as "overwrite" to overwrite the existing path.

In [24]:
d2.write.format("parquet").mode("overwrite").save("/content/parquet_files/Previsao.parquet")

AnalysisException: Since Spark 2.3, the queries from raw JSON/CSV files are disallowed when the
referenced columns only include the internal corrupt record column
(named _corrupt_record by default). For example:
spark.read.schema(schema).csv(file).filter($"_corrupt_record".isNotNull).count()
and spark.read.schema(schema).csv(file).select("_corrupt_record").show().
Instead, you can cache or save the parsed results and then send the same query.
For example, val df = spark.read.schema(schema).csv(file).cache() and then
df.filter($"_corrupt_record".isNotNull).count().

In [25]:
parsed.first()

Row(id_1=3148, id_2=8326, cmp_fname_c1=1.0, cmp_fname_c2=None, cmp_lname_c1=1.0, cmp_lname_c2=None, cmp_sex=1, cmp_bd=1, cmp_bm=1, cmp_by=1, cmp_plz=1, is_match=True)

In [26]:
parsed.count()

5749132

In [ ]:
parsed.colletc()

In [ ]:
parsed.write.format("parquet").("content/parquet_files/Previsao.parquet")

In [30]:
parsed.printSchema()

root
 |-- id_1: integer (nullable = true)
 |-- id_2: integer (nullable = true)
 |-- cmp_fname_c1: double (nullable = true)
 |-- cmp_fname_c2: double (nullable = true)
 |-- cmp_lname_c1: double (nullable = true)
 |-- cmp_lname_c2: double (nullable = true)
 |-- cmp_sex: integer (nullable = true)
 |-- cmp_bd: integer (nullable = true)
 |-- cmp_bm: integer (nullable = true)
 |-- cmp_by: integer (nullable = true)
 |-- cmp_plz: integer (nullable = true)
 |-- is_match: boolean (nullable = true)



In [31]:
parsed.show(5)

+-----+-----+------------+------------+------------+------------+-------+------+------+------+-------+--------+
| id_1| id_2|cmp_fname_c1|cmp_fname_c2|cmp_lname_c1|cmp_lname_c2|cmp_sex|cmp_bd|cmp_bm|cmp_by|cmp_plz|is_match|
+-----+-----+------------+------------+------------+------------+-------+------+------+------+-------+--------+
| 3148| 8326|         1.0|        NULL|         1.0|        NULL|      1|     1|     1|     1|      1|    true|
|14055|94934|         1.0|        NULL|         1.0|        NULL|      1|     1|     1|     1|      1|    true|
|33948|34740|         1.0|        NULL|         1.0|        NULL|      1|     1|     1|     1|      1|    true|
|  946|71870|         1.0|        NULL|         1.0|        NULL|      1|     1|     1|     1|      1|    true|
|64880|71676|         1.0|        NULL|         1.0|        NULL|      1|     1|     1|     1|      1|    true|
+-----+-----+------------+------------+------------+------------+-------+------+------+------+-------+--

In [32]:
parsed.count()

5749132

In [33]:
parsed.cache()

DataFrame[id_1: int, id_2: int, cmp_fname_c1: double, cmp_fname_c2: double, cmp_lname_c1: double, cmp_lname_c2: double, cmp_sex: int, cmp_bd: int, cmp_bm: int, cmp_by: int, cmp_plz: int, is_match: boolean]

In [34]:
from pyspark.sql.functions import col


parsed.groupBy("is_match").count().orderBy(col("count").desc()).show()

+--------+-------+
|is_match|  count|
+--------+-------+
|   false|5728201|
|    true|  20931|
+--------+-------+



In [35]:
from pyspark.sql.functions import avg, stddev


parsed.agg(avg("cmp_sex"), stddev("cmp_sex")).show()

+-----------------+-------------------+
|     avg(cmp_sex)|    stddev(cmp_sex)|
+-----------------+-------------------+
|0.955001381078048|0.20730111116897443|
+-----------------+-------------------+



In [36]:
parsed.createOrReplaceTempView("linkage")

In [37]:
spark.sql("""
  SELECT is_match, COUNT(*) AS cnt
  FROM linkage
  GROUP BY is_match
  ORDER BY cnt DESC
""").show()

+--------+-------+
|is_match|    cnt|
+--------+-------+
|   false|5728201|
|    true|  20931|
+--------+-------+



In [38]:
spark_session = SparkSession.builder.master("local[4]").enableHiveSupport().getOrCreate()